In [1]:
import os
import re
import pandas as pd
from pathlib import Path
import sys # Import sys for exit

# --- Configuration ---
# Set the directory containing your SRT files
SRT_DIRECTORY = Path("./srt_files")  # <--- CHANGE THIS to your folder path

# Set the directory where the output Excel files will be saved
OUTPUT_DIRECTORY = Path("./output_ratings") # <--- CHANGE THIS if desired

# --- Script Logic ---

def parse_srt_file(file_path):
    """
    Parses an SRT file to extract turn ID, start time, end time, and rating
    for each subtitle block where the text is a rating in single quotes.

    Args:
        file_path (Path): The path to the SRT file.

    Returns:
        list: A list of dictionaries, each containing 'Turn ID', 'Start Time',
              'End Time', and 'Rating' for a subtitle block. Returns an empty
              list if errors occur or no matching blocks are found.
    """
    extracted_data = []
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()

            # Regex to find SRT blocks with a rating ('number') as the text
            # Group 1: Turn ID (\d+)
            # Group 2: Start Time (\d{2}:\d{2}:\d{2},\d{3})
            # Group 3: End Time (\d{2}:\d{2}:\d{2},\d{3})
            # Group 4: Rating (\d+) inside single quotes
            # Handles potential whitespace variations
            pattern = re.compile(
                r"(\d+)\s*\n"                                      # Turn ID line
                r"(\d{2}:\d{2}:\d{2},\d{3})\s*-->\s*(\d{2}:\d{2}:\d{2},\d{3})\s*\n" # Timestamp line
                r"'(\d+)'"                                         # Rating line ('number')
                # r"(.+?)\n\n", re.DOTALL # Alternative if text isn't *just* the rating
            )

            matches = pattern.findall(content)

            for match in matches:
                turn_id, start_time, end_time, rating = match
                extracted_data.append({
                    'Turn ID': int(turn_id), # Store as integer
                    'Start Time': start_time,
                    'End Time': end_time,
                    'Rating': int(rating) # Store rating as integer
                })

    except FileNotFoundError:
        print(f"Error: File not found - {file_path}")
    except Exception as e:
        print(f"Error processing file {file_path}: {e}")
    
    return extracted_data

# --- Main Execution ---

# Ensure the input directory exists
if not SRT_DIRECTORY.is_dir():
    print(f"Error: Input directory not found - {SRT_DIRECTORY}")
    print("Please create the directory and place your SRT files inside, or update the SRT_DIRECTORY path.")
    sys.exit(1) # Use sys.exit for a cleaner exit

# Create the output directory if it doesn't exist
try:
    OUTPUT_DIRECTORY.mkdir(parents=True, exist_ok=True)
except Exception as e:
    print(f"Error: Could not create output directory - {OUTPUT_DIRECTORY}: {e}")
    sys.exit(1)


# Get SRT files and sort them for consistent order
srt_files = sorted(list(SRT_DIRECTORY.glob("*.srt")))

if not srt_files:
    print(f"No .srt files found in directory: {SRT_DIRECTORY}")
    sys.exit(0) # No files is not an error, just nothing to do

print(f"Found {len(srt_files)} SRT files in {SRT_DIRECTORY}. Processing...")

processed_count = 0
error_count = 0

for srt_file in srt_files:
    print(f"\nProcessing: {srt_file.name}")
    
    # Parse the current SRT file
    turn_data = parse_srt_file(srt_file)

    if not turn_data:
        print(f"  No valid rating entries found or error processing file: {srt_file.name}")
        error_count += 1
        continue # Skip to the next file

    # Create a DataFrame for this file's data
    df = pd.DataFrame(turn_data)
    
    # Define the output Excel filename based on the input SRT filename
    # Example: input 'personA.srt' -> output 'personA_ratings.xlsx'
    output_filename = OUTPUT_DIRECTORY / f"{srt_file.stem}_ratings.xlsx" 

    # --- Save to Excel ---
    try:
        # index=False prevents writing the DataFrame index as a column
        # engine='openpyxl' is required for .xlsx format
        df.to_excel(output_filename, index=False, engine='openpyxl') 
        print(f"  Successfully saved data to: {output_filename}")
        processed_count += 1
    except Exception as e:
        print(f"  Error saving Excel file {output_filename}: {e}")
        print("  Make sure you have 'openpyxl' installed (`pip install openpyxl`)")
        error_count += 1

print(f"\n--- Processing Complete ---")
print(f"Successfully processed and saved data for {processed_count} files.")
if error_count > 0:
    print(f"Encountered errors or found no data in {error_count} files.")

Found 39 SRT files in srt_files. Processing...

Processing: P10_1_annotation_EL.srt
  Successfully saved data to: output_ratings/P10_1_annotation_EL_ratings.xlsx

Processing: P11_1_annotation_EL.srt
  Successfully saved data to: output_ratings/P11_1_annotation_EL_ratings.xlsx

Processing: P12_1_annotation_EL.srt
  Successfully saved data to: output_ratings/P12_1_annotation_EL_ratings.xlsx

Processing: P13_1_annotation_EL.srt
  Successfully saved data to: output_ratings/P13_1_annotation_EL_ratings.xlsx

Processing: P14_1_annotation_EL.srt
  Successfully saved data to: output_ratings/P14_1_annotation_EL_ratings.xlsx

Processing: P15_1_annotation_EL.srt
  Successfully saved data to: output_ratings/P15_1_annotation_EL_ratings.xlsx

Processing: P16_1_annotation_EL.srt
  Successfully saved data to: output_ratings/P16_1_annotation_EL_ratings.xlsx

Processing: P17_1_annotation_EL.srt
  Successfully saved data to: output_ratings/P17_1_annotation_EL_ratings.xlsx

Processing: P18_1_annotation_EL.

In [3]:
import pandas as pd
import os

# Path to the Excel file
excel_file_path = 'user-self-reports/target-enjoyment.xlsx'

# Import the target-enjoyment tab
target_enjoyment_df = pd.read_excel(excel_file_path, sheet_name='target-enjoyment', header=1)

# Display the first few rows to verify
print(target_enjoyment_df.head())

os.makedirs("exchange-data", exist_ok=True)

for person in target_enjoyment_df["PID"]:
    first_interaction_robot = target_enjoyment_df.loc[target_enjoyment_df["PID"] == person, "Q1-Robot"].values[0]
    
    transcript_path = f"11l-corrected-transcripts/audio-{first_interaction_robot}-P{person}.txt"
    with open(transcript_path, "r") as transcript:
        transcript_lines = transcript.readlines()

        exchanges = {}
        utterance_index = 0
        for line in transcript_lines:
            if (not "[Robot]" in line) and (not "[User]" in line):
                if line.strip() == "":
                    continue
                else:
                    exchanges[utterance_index][2] += line.strip() + " "
            else:
                utterance_index += 1

                time_part = line.split("[")[0].strip()  # Get "00:00:21,459 --> 00:00:25,439"
                start_time, end_time = time_part.split("-->")
                start_time = start_time.strip()  # "00:00:21,459"
                end_time = end_time.strip()  # "00:00:25,439"
                if "[Robot]" in line:
                    exchanges[utterance_index] = [start_time, end_time, "Robot: "]
                else:
                    exchanges[utterance_index] = [start_time, end_time, "User: "]
    # Create a DataFrame from the exchange data
    exchange_df = pd.DataFrame(exchanges).T
    exchange_df.columns = ["Start Time", "End Time", "Utterance"]
    # Save the DataFrame to an Excel file
    # exchange_df.to_excel(f"exchange-data/exchange_data-P{person}-{first_interaction_robot}.xlsx", index=False)

   PID  Condition Q1-Robot Q2-Robot  Q1-1  Q1-2  Q2-1  Q2-2  Q3-1  Q3-2  Q4-1  \
0    4          3    Alice    Clara     3     5     5     6     2     5     2   
1    5          2    Clara    Alice     6     6     6     5     6     5     6   
2    6          4    Clara    Alice     6     5     7     6     7     3     6   
3    7          2    Clara    Alice     7     7     7     7     7     7     7   
4    8          3    Alice    Clara     7     7     6     7     6     7     6   

   Q4-2  Q5-1  Q5-2  Q6-1  Q6-2  Q7-1  Q7-2  
0     5     5     6     5     6     2     1  
1     5     7     6     6     4     1     2  
2     5     7     4     6     6     1     1  
3     7     7     7     7     7     1     1  
4     7     6     6     7     7     1     1  
